In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import torch

import hashlib
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS

In [ ]:
# YOLOv5 레포지토리 클론
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
# 필요한 라이브러리 설치
!pip install -U -r yolov5/requirements.txt

In [ ]:
model = torch.hub.load('yolov5', 'custom', path='/content/drive/MyDrive/Capstone/runs/train/exp/weights/best.pt', source='local')

YOLOv5 🚀 v7.0-318-gc0380fd8 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
!apt-get install libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import cv2
import numpy as np
import hashlib
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS

# 타겟 원본 이미지 메타데이터에서 생성 날짜 추출 #
def get_image_creation_date(image_path):
    with Image.open(image_path) as img:
        exif_data = img._getexif()
        if exif_data is not None:
            for tag_id in exif_data:
                tag_name = TAGS.get(tag_id, tag_id)
                if tag_name == "DateTimeOriginal":
                    return exif_data[tag_id]
        date_input = input("이미지의 생성 날짜와 시간을 YYYY-MM-DD HH:MM:SS 형식으로 입력해주세요: ") # 실제로는 메타데이터로 받아옴
        return datetime.strptime(date_input, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')

# 디지털 워터마크 생성 및 LSB 스테가노그래피 적용 #
def apply_watermark_to_bbox(image, bbox, watermark):
    x_min, y_min, x_max, y_max, confidence, class_ = bbox

    # 바운딩 박스의 너비와 높이 계산
    bbox_width = x_max - x_min
    bbox_height = y_max - y_min

    # 워터마크를 바운딩 박스의 크기에 맞게 조정
    scaled_watermark = cv2.resize(watermark, (bbox_width, bbox_height))

    # 바운딩 박스의 시작과 끝 좌표 계산
    start_x = max(x_min, 0)
    start_y = max(y_min, 0)
    end_x = min(x_max, image.shape[1])
    end_y = min(y_max, image.shape[0])

    # 바운딩 박스의 유효한 부분에 워터마크 적용
    for i in range(start_y, end_y):
        for j in range(start_x, end_x):
            pixel = image[i, j]
            watermark_pixel = scaled_watermark[i - start_y, j - start_x]
            if watermark_pixel.any() != 0:  # 워터마크 픽셀 값이 0이 아닌 경우에만 적용
                image[i, j] = watermark_pixel

    return image

def embed_watermark(author_name, image_path, output_path):
    creation_date = get_image_creation_date(image_path) # 원본 이미지 생성 날짜 추출 함수 호출
    application_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S') # 디지털 워터마크 적용 날짜 및 시간 추출
    watermark_data = f"{author_name}|{creation_date}|{application_date}" # main()에서 입력받은 저작권자명 포함 watermark_data 생성

    img = cv2.imread(image_path, cv2.IMREAD_COLOR) # 지정된 파일 경로에서 컬러 이미지 읽기
    bboxes = [(100, 100, 200, 200, 0.9, 'person')]  # 바운딩 박스 정보 (임시로 예시를 넣었습니다)
    watermark = np.random.randint(0, 2, (50, 50, 3), dtype=np.uint8) * 255  # 임의의 워터마크 생성 (50x50 크기의 컬러 이미지)

    for bbox in bboxes:
        img = apply_watermark_to_bbox(img, bbox, watermark) # 바운딩 박스에 워터마크 적용

    cv2.imwrite(output_path, img) # 최종적으로 워터마크가 적용된 이미지를 사용자가 지정한 경로(output_path)에 저장

# 사용자 입력과 프로그램 실행 #
def main():
    author_name = input("저작권자명을 입력해주세요: ")
    image_path = input("대상 이미지 파일 경로를 입력해주세요: ")
    output_path = input("워터마크가 적용된 이미지를 저장할 경로를 입력해주세요(확장자 포함): ")
    embed_watermark(author_name, image_path, output_path)
    print("워터마크 적용이 완료되었습니다.")

if __name__ == "__main__":
    main()


저작권자명을 입력해주세요: KIM GA-IN
대상 이미지 파일 경로를 입력해주세요: /content/drive/MyDrive/Capstone/example/1-1.png
워터마크가 적용된 이미지를 저장할 경로를 입력해주세요(확장자 포함): /content/drive/MyDrive/Capstone/result/1-1-test2.png
이미지의 생성 날짜와 시간을 YYYY-MM-DD HH:MM:SS 형식으로 입력해주세요: 2024-02-02 02:02:02
워터마크 적용이 완료되었습니다.
